# High Performance

mask = (x > 0.5) & (y < 0.5)

Use the mask to filter the values

example:
df = df[df[mask]]

#Intermediate variables in memory
```python
We get a tmp1: (x > 0.5) tmp is a temporary
And a tmp2: (y < 0.5)
mask = tmp1 + tmp2
```

Can use pd.eval("") -> Performance elementwise directly using numexpr
Good for compound expressions (for example several &)

In [1]:
import pandas as pd
import numpy as np

In [9]:
nrows, ncols = 1000000, 100
df1, df2, df3, df4 = [pd.DataFrame(np.random.randn(nrows, ncols)) for _ in range(4)]
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.240306,-0.221773,0.753270,-3.064099,-0.617434,2.597266,-0.432093,0.386695,1.049083,-0.829530,...,0.104370,-0.125029,-0.247992,0.006140,1.014138,0.858381,0.528590,0.452912,0.149254,0.051754
1,-0.680475,0.456278,0.631022,0.743280,1.455826,1.939900,0.090809,-1.104869,0.782235,1.678255,...,1.247761,0.234729,-0.103178,-0.772522,-0.753366,-0.382640,1.018128,1.334504,0.353645,-0.609486
2,-0.117242,2.527464,0.173418,-0.397872,-0.408325,-0.943289,-0.323200,-1.393180,-0.102970,-1.594512,...,-0.657065,-0.613723,0.504816,-0.329243,0.017735,0.467915,0.560815,0.931082,0.645865,0.128262
3,-0.203879,-0.528676,-1.132364,0.879731,1.180326,0.436551,0.789410,1.075231,0.400651,0.054151,...,-0.393466,-0.551187,0.579732,-2.147053,-1.219839,0.987889,0.924658,-0.391259,0.913869,-1.183981
4,-0.108698,0.799664,-1.260508,1.134219,1.701136,-0.953404,-1.214413,0.734029,0.325781,0.176139,...,-0.193483,-0.305260,-0.434593,-0.711460,-0.651663,0.878559,0.046587,0.202285,-0.381191,1.270371


In [10]:
%timeit df1 + df2 + df3 + df4

1.33 s ± 164 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%timeit pd.eval("df1 + df2 + df3 + df4") #pd.eval is much faster

552 ms ± 73.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
plain = df1 + df2 + df3 + df4
sum_eval = pd.eval("df1 + df2 + df3 + df4")

sum_eval.equals(plain) #Are they the same? Yes!

True

In [16]:
#df.eval() #We use eval directlt on the dataframe
rolls = pd.DataFrame(np.random.randint(1,6, (6,3)), columns = ["Die1", "Die2", "Die3"])
rolls.eval("Sum = Die1 + Die2 + Die3", inplace=True)
rolls

,Die1,Die2,Die3,Sum
0,2,3,3,8
1,5,3,1,9
2,1,4,5,10
3,4,4,2,10
4,1,2,1,4
5,3,4,5,12


In [17]:
#use variables
high = 10
rolls.eval("Winner = Sum > @high", inplace=True) #With @ we reach local variables in the same scope
rolls

,Die1,Die2,Die3,Sum,Winner
0,2,3,3,8,False
1,5,3,1,9,False
2,1,4,5,10,False
3,4,4,2,10,False
4,1,2,1,4,False
5,3,4,5,12,True


In [20]:
#Filter out the traditional way
rolls[rolls["Sum"] <= high]

,Die1,Die2,Die3,Sum,Winner
0,2,3,3,8,False
1,5,3,1,9,False
2,1,4,5,10,False
3,4,4,2,10,False
4,1,2,1,4,False


## Query

Gives a new dataframe, (eval is used to generate new rows)

Both query and eval is a bit slower for small datasets, but it is much easier to read (so use it!)

In [21]:
rolls.query("Sum <= @high")

,Die1,Die2,Die3,Sum,Winner
0,2,3,3,8,False
1,5,3,1,9,False
2,1,4,5,10,False
3,4,4,2,10,False
4,1,2,1,4,False


In [15]:
os = pd.read_csv("../Data/athlete_events.csv")
os.head(2)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN


In [18]:
%timeit os[os["NOC"] == "SWE"]
%timeit os.query("NOC == 'SWE'")

27.2 ms ± 3.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
11.7 ms ± 293 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%timeit os[os["Height"] > 180]
%timeit os.query("Height > 180") #In this case query is a bit slower

13 ms ± 1.09 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
15.5 ms ± 185 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [20]:
%timeit os[(os["Sex"] == "F") & (os["Height"] > 180 & (os["NOC"] == "SWE"))]
%timeit os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'")

57.8 ms ± 3.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
16.7 ms ± 364 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
os.query("Sex == 'F' & Height > 180 & NOC == 'SWE'")

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
729,417,Sara Helena berg,F,17.0,190.0,73.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Swimming,Swimming Women's 50 metres Freestyle,NaN
5175,2940,Jenny Alm,F,27.0,184.0,80.0,Sweden,SWE,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Women's Handball,NaN
7555,4210,Marina Vladimirovna Andrievskaia,F,29.0,182.0,66.0,Sweden,SWE,2004 Summer,2004,Summer,Athina,Badminton,Badminton Women's Singles,NaN
19070,10088,Anna Therese Bengtsson,F,29.0,187.0,83.0,Sweden,SWE,2008 Summer,2008,Summer,Beijing,Handball,Handball Women's Handball,NaN
28221,14643,Maria Helene Brandin,F,25.0,186.0,85.0,Sweden,SWE,1988 Summer,1988,Summer,Seoul,Rowing,Rowing Women's Double Sculls,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242230,121329,Linnea Maria Torstenson,F,33.0,186.0,82.0,Sweden,SWE,2016 Summer,2016,Summer,Rio de Janeiro,Handball,Handball Women's Handball,NaN
259242,129789,Anna Karolina Westberg,F,22.0,184.0,78.0,Sweden,SWE,2000 Summer,2000,Summer,Sydney,Football,Football Women's Football,NaN
259243,129789,Anna Karolina Westberg,F,26.0,184.0,78.0,Sweden,SWE,2004 Summer,2004,Summer,Athina,Football,Football Women's Football,NaN
259934,130126,Johanna Maria Wiberg,F,24.0,184.0,78.0,Sweden,SWE,2008 Summer,2008,Summer,Beijing,Handball,Handball Women's Handball,NaN
